In [ ]:
using Gadfly, RDatasets
import ForwardDiff: gradient

### Sanity check

In [ ]:
f(x) = sum(x)*log(abs(sum(x)))
f([2,5])

In [ ]:
gradient(f, [2,5])

### Generic gradient descent

In [ ]:
function GradientDescent(L, x0; rate=.1, N=100, eps=.001)
    for i = 1:N 
        ∇L = gradient(L, x0)
        if norm(∇L) < eps
            break
        end
        x0 -= rate * ∇L
    end
    x0
end

### Curve fitting

In [ ]:
x = rand(20) * 6
w = [.5; -2.; 1]
y = [x.^2  x  ones(x)] * w  + rand(size(x)) * .2
#         ^^^                      ^^^
#   2nd degree polynomial       Noisy data

plot(
    layer(x=x, y=y, Geom.point),
    layer(x -> dot([x^2; x; 1], w), 0, 6, Theme(default_color=colorant"green")),
    Guide.title("Ground truth: w = $w"))

In [ ]:
esw = GradientDescent([1,1,1], rate=.001, N=5000) do w
    mean(([x.^2  x  ones(x)] * w - y).^2)
end

plot(
    layer(x=x, y=y, Geom.point),
    layer(x -> dot([x^2; x; 1], w), 0, 6, Theme(default_color=colorant"green")),
    layer(x -> dot([x^2; x; 1], esw), 0, 6, Theme(default_color=colorant"red")),
    Guide.title("Estimate: w = $esw"))

### Iris dataset

In [ ]:
iris = dataset("datasets", "iris")
plot(iris, x=:SepalWidth, y=:SepalLength, color=:Species) |> display

In [ ]:
X = convert(Array, iris[[:SepalWidth, :SepalLength, :PetalWidth]])
y = convert(Array{Int64}, iris[:Species] .== "setosa");

### Logistic Regression

In [ ]:
logit(x,w) = 1 / (1 + exp(-dot([x; 1], w)))

logisticRegression(X,y) =
    GradientDescent(rand(size(X,2)+1)) do w
        e = 0 # Loss is negative log likelihood
        for i = 1:size(X,1) # <<< A loop!
            if y[i] == 0    # << Branches!
                e += log(1-logit(X[i,:], w))
            else
                e += log(logit(X[i,:], w))
            end
        end
        -e/size(X,1)
    end

In [ ]:
w = logisticRegression(X,y)
show(w)
pred = [logit(X[i,:], w) > .5 for i = 1:size(X,1)]
plot(x=X[:,1], y=X[:,2], color=pred)

### Bagging Gradient Descent

In [ ]:
baggingLogisticRegression(X,y) =
    GradientDescent(rand(size(X,2)+1)) do w
        e = 0
        for k = 1:size(X,1)*.6
        
            i = rand(1:size(X,1)) # <<< Random state!
    
            if y[i] == 0
                e += log(1-logit(X[i,:], w))
            else
                e += log(logit(X[i,:], w))
            end
        end
        -e/(size(X,1)*.6)
    end

In [ ]:
w = baggingLogisticRegression(X,y)
show(w)
pred = [logit(X[i,:], w) > .5 for i = 1:size(X,1)]
plot(x=X[:,1], y=X[:,2], color=pred)